# This is a demo + test notebook for GraphTranslatorModule

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from GraphTranslatorModule import GraphTranslatorModule
from pytorch_lightning import Trainer
import torch
from torch.nn import functional as F

## A. This section is a primer for LightningModule

### This is a lightning module instead of a torch.nn.Module. A LightningModule is equivalent to a pure PyTorch Module except it has added functionality. However, you can use it EXACTLY the same as you would a PyTorch Module.

In [ ]:
net = GraphTranslatorModule()
x = torch.randn(1, 1, 3, 3)
out = net(x)

In [ ]:
from pl_bolts.datasets import DummyDataset
trainer = Trainer(max_epochs=3)
trainer.fit(net, torch.utils.data.DataLoader(DummyDataset((1,3,3),(1,), num_samples=100), num_workers=8))

In [ ]:
trainer.test(net, torch.utils.data.DataLoader(DummyDataset((1,3,3),(1,), num_samples=10), num_workers=8))